# 4. Neural Style Transfer on AKS

Now that the AKS cluster is up, we need to deploy our __flask app__ and __scoring app__ onto it.

To do so, we'll do the following:
1. Build our __flask app__ and __scoring app__ push it to Dockerhub
2. Create our dot-yaml files for each of these apps (these dot-yaml files will need to have the proper configuration for the pods to use blobfuse to access our blob storage container). We should end up creating: `flask_app_deployment.json` and `scoring_app_deployment.json`
3. Use `kubectl` to make these deployments to our AKS cluster
4. Expose the __flask app__ REST endpoint so that it can be accessed externally

### Kubernetes Deployment
In this notebook, we will deploy our __flask app__ and __scoring app__ on the kubernetes cluster. Since the __flask app__ does not require heavy computation, we will deploy it on one node and reserve the remaining nodes for the __scoring app__ as it will perform the parallel computation.

---

### Import packages and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import subprocess
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Build Scoring App Docker Image

In [3]:
%%writefile scoring_app/requirements.txt
azure==4.0.0
torch==0.4.1
torchvision==0.2.1

Writing scoring_app/requirements.txt


In [4]:
%%writefile scoring_app/Dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.6
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
ENV PYTHONPATH /code/:$PYTHONPATH

RUN mkdir /app
WORKDIR /app
ADD process_images_from_queue.py /app
ADD style_transfer.py /app
ADD main.py /app
ADD util.py /app
ADD requirements.txt /app
ADD constants.py /app

RUN pip install --no-cache-dir -r requirements.txt

RUN rm /opt/conda/envs/py3.6/lib/python3.6/site-packages/azure/servicebus/constants.py
RUN cp constants.py /opt/conda/envs/py3.6/lib/python3.6/site-packages/azure/servicebus/

CMD ["python", "main.py"]

Writing scoring_app/Dockerfile


In [5]:
!sudo docker build -t {get_key(env_path, "SCORING_IMAGE")} scoring_app

Sending build context to Docker daemon  37.89kB
Step 1/20 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> 7e8410ba243b
Step 2/20 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Using cache
 ---> 2aee8c10151f
Step 3/20 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates         cmake         curl         git         nginx         supervisor         wget &&         rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c69b13e821b7
Step 4/20 : ENV PYTHON_VERSION=3.6
 ---> Using cache
 ---> 9c3490d15c2d
Step 5/20 : RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  &&     chmod +x ~/miniconda.sh &&     ~/miniconda.sh -b -p /opt/conda &&     rm ~/miniconda.sh &&     /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION &&     /opt/conda/bin/conda c

Tag and push docker image

In [6]:
!sudo docker login --username pjh177787 --password 'Hans&951022'

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/aperture/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
repo = "{}/{}".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "SCORING_IMAGE"))

In [8]:
!sudo docker tag {get_key(env_path, "SCORING_IMAGE")} {repo}

In [9]:
!sudo docker push {repo}

The push refers to repository [docker.io/pjh177787/cambridgesp]

ea68a8e4: Preparing 
e9c72ee2: Preparing 
4636511d: Preparing 
30360c47: Preparing 
112a59d2: Preparing 
b59fe06d: Preparing 
93a511e9: Preparing 
34dc443b: Preparing 
711df71e: Preparing 
0c6970e5: Preparing 
e82df0c2: Preparing 
89b11fc2: Preparing 
b7e6614f: Preparing 
c5838665: Preparing 
7ea5d26b: Preparing 
7acf624e: Preparing 
a0fe4fdd: Preparing 
1c46eb92: Preparing 
6e800c43: Preparing 
f22d44f3: Preparing 
6f329a25: Preparing 
7de5faec: Preparing 
a27b0484: Mounted from pjh177787/oxford_scoring_app latest: digest: sha256:a3173586db425c21be1f7fa176b4fdf0fcae5c5a435dca60672b4b1e24468001 size: 5129


### Build Flask App Docker Image

Create our Dockerfile and save it to the directory, `flask_app/`.

In [10]:
%%writefile flask_app/Dockerfile

FROM continuumio/miniconda3

RUN mkdir /app
WORKDIR /app
ADD add_images_to_queue.py /app
ADD preprocess.py /app
ADD postprocess.py /app
ADD util.py /app
ADD main.py /app
ADD constants.py /app

RUN conda install -c conda-forge -y ffmpeg
RUN pip install azure
RUN pip install flask

RUN rm /opt/conda/lib/python3.7/site-packages/azure/servicebus/constants.py
RUN cp constants.py /opt/conda/lib/python3.7/site-packages/azure/servicebus/

CMD ["python", "main.py"]

Writing flask_app/Dockerfile


Build the Docker image

In [11]:
!sudo docker build -t {get_key(env_path, "FLASK_IMAGE")} flask_app

Sending build context to Docker daemon  26.11kB
Step 1/15 : FROM continuumio/miniconda3
 ---> 6b5cf97566c3
Step 2/15 : RUN mkdir /app
 ---> Using cache
 ---> 3e41fbdb3278
Step 3/15 : WORKDIR /app
 ---> Using cache
 ---> 032cc5cbe3da
Step 4/15 : ADD add_images_to_queue.py /app
 ---> Using cache
 ---> a7aad3e80c4e
Step 5/15 : ADD preprocess.py /app
 ---> Using cache
 ---> 88ec5fb7fcf2
Step 6/15 : ADD postprocess.py /app
 ---> Using cache
 ---> 796d0a2feade
Step 7/15 : ADD util.py /app
 ---> Using cache
 ---> 842a56dcdc61
Step 8/15 : ADD main.py /app
 ---> Using cache
 ---> c01786b74db0
Step 9/15 : ADD constants.py /app
 ---> Using cache
 ---> 768f91f78a6d
Step 10/15 : RUN conda install -c conda-forge -y ffmpeg
 ---> Using cache
 ---> 063cda8ad60a
Step 11/15 : RUN pip install azure
 ---> Using cache
 ---> 8657da9773a1
Step 12/15 : RUN pip install flask
 ---> Using cache
 ---> 9413b1052c11
Step 13/15 : RUN rm /opt/conda/lib/python3.7/site-packages/azure/servicebus/constants.py
 ---> Using 

Tag and push.

In [12]:
repo = "{}/{}".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "FLASK_IMAGE"))

In [13]:
!sudo docker tag {get_key(env_path, "FLASK_IMAGE")} {repo}

In [14]:
!sudo docker push {repo}

The push refers to repository [docker.io/pjh177787/cambridgefp]

628240c8: Preparing 
8b70bd2c: Preparing 
5bfaeb6f: Preparing 
a73a5bf7: Preparing 
0060ed1e: Preparing 
f4f6db82: Preparing 
2767c2e6: Preparing 
a5e2b056: Preparing 
fe3428e9: Preparing 
55a10f32: Preparing 
18d16c6e: Preparing 
66549fba: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
41a8b943: Mounted from pjh177787/oxford_flask_app latest: digest: sha256:16c57acc1b0fa990eaa236a0190ba760c8bf4b9112c2ed22274db8888bdd779e size: 3870


### Create our Flask App and Scoring App deployments on AKS

We need to deploy both our aci and aks docker images to the AKS cluster. Since we'll need to set up our gpu and drivers and blobfuse mount point for both deployments, we'll set these up first:

In [15]:
volume_mounts = [
#     {"name": "nvidia", "mountPath": "/usr/local/nvidia"},
    {"name": "blob", "mountPath": get_key(env_path, "MOUNT_DIR")},
]

resources = {
#     "requests": {"alpha.kubernetes.io/nvidia-gpu": 1},
#     "limits": {"alpha.kubernetes.io/nvidia-gpu": 1},
}

volumes = [
#     {"name": "nvidia", "hostPath": {"path": "/usr/local/nvidia"}},
    {
        "name": "blob",
        "flexVolume": {
            "driver": "azure/blobfuse",
            "readOnly": False,
            "secretRef": {"name": "blobfusecreds"},
            "options": {
                "container": get_key(env_path, "STORAGE_CONTAINER_NAME"),
                "tmppath": "/tmp/blobfuse",
                "mountoptions": "--file-cache-timeout-in-seconds=120 --use-https=true",
            },
        },
    },
]

env = [
    {
        "name": "MOUNT_DIR", 
        "value": get_key(env_path, "MOUNT_DIR")
    },
    {
        "name": "LB_LIBRARY_PATH",
        "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib",
    },
    {
        "name": "DP_DISABLE_HEALTHCHECKS", 
        "value": "xids"
    },
    {
        "name": "STORAGE_MODEL_DIR",
        "value": get_key(env_path, "STORAGE_MODEL_DIR")
    },
    {
        "name": "SUBSCRIPTION_ID",
        "value": get_key(env_path, "SUBSCRIPTION_ID")
    },
    {
        "name": "RESOURCE_GROUP",
        "value": get_key(env_path, "RESOURCE_GROUP")
    },
    {
        "name": "REGION",
        "value": get_key(env_path, "REGION")
    },
    {
        "name": "SB_SHARED_ACCESS_KEY_NAME",
        "value": get_key(env_path, "SB_SHARED_ACCESS_KEY_NAME")
    },
    {
        "name": "SB_SHARED_ACCESS_KEY_VALUE",
        "value": get_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE")
    },
    {
        "name": "SB_NAMESPACE",
        "value": get_key(env_path, "SB_NAMESPACE")
    },
    {
        "name": "SB_QUEUE", 
        "value": get_key(env_path, "SB_QUEUE")
    },
]

Define the aks deployment and save it to a `scoring_app_deployment.json` file using the variables set above.

In [16]:
scoring_app_deployment_json = {
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "scoring-app", 
        "labels": {
            "purpose": "dequeue_messages_and_apply_style_transfer"
        }
    },
    "spec": {
        "replicas": int(get_key(env_path, "NODE_COUNT")) - 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "scoring-app"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "scoring-app",
                        "image": "{}/{}:latest".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "SCORING_IMAGE")),
                        "volumeMounts": volume_mounts,
                        "resources": resources,
                        "ports": [{
                            "containerPort": 433
                        }],
                        "env": env,
                    }
                ],
                "volumes": volumes
            },
        },
    },
}

with open("scoring_app_deployment.json", "w") as outfile:
    json.dump(scoring_app_deployment_json, outfile, indent=4, sort_keys=True)
    outfile.write('\n\n')

Using the `scoring_app_deployment.json` we created, create our deployment on AKS. This can take a few minutes...

In [17]:
!kubectl delete -f scoring_app_deployment.json

Error from server (NotFound): error when deleting "scoring_app_deployment.json": deployments.apps "scoring-app" not found


In [18]:
!kubectl create -f scoring_app_deployment.json

deployment.apps/scoring-app created


Define the flask app deployment and save it to a `flask_app_deployment.json` file using the variables set above.

In [19]:
flask_app_deployment_json = {
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "flask-app", 
        "labels": {
            "purpose": "pre_and_post_processing_and_queue_images"
        }
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "flask-app"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "flask-app",
                        "image": "{}/{}:latest".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "FLASK_IMAGE")),
                        "volumeMounts": volume_mounts,
                        "resources": resources,
                        "ports": [{
                            "containerPort": 8080
                        }],
                        "env": env,
                    }
                ],
                "volumes": volumes
            },
        },
    },
}

with open("flask_app_deployment.json", "w") as outfile:
    json.dump(flask_app_deployment_json, outfile, indent=4, sort_keys=True)
    outfile.write('\n\n')

Using the `flask_app_deployment.json` we created, create our flask app deployment on AKS. This can take a few minutes...

In [20]:
!kubectl delete -f flask_app_deployment.json

Error from server (NotFound): error when deleting "flask_app_deployment.json": deployments.apps "flask-app" not found


In [21]:
!kubectl create -f flask_app_deployment.json

deployment.apps/flask-app created


These deployments may take a few minutes. You can inspect the state of the pods by running the command: `kubectl get pods`. When the deployment is done, the results may look as follows:
```bash
NAME                           READY   STATUS              RESTARTS   AGE
flask-app-6db66c97ff-x8rq4     1/1     Running             0          78s
scoring-app-846dd6bc79-5nm5b   1/1     Running             0          73s
scoring-app-846dd6bc79-6qc6k   1/1     Running             0          73s
scoring-app-846dd6bc79-8gtsv   1/1     Running             0          73s
scoring-app-846dd6bc79-hjsfc   1/1     Running             0          73s
```

In [35]:
!kubectl get pods

NAME                          READY   STATUS              RESTARTS   AGE
aks-ssh-589f4659c5-rdq97      1/1     Running             0          3h9m
flask-app-6575466bf-lc4g5     1/1     Running             0          3m32s
scoring-app-94fdf55c8-2k7dk   0/1     ContainerCreating   0          3m43s
scoring-app-94fdf55c8-6tppl   0/1     ImagePullBackOff    0          3m43s
scoring-app-94fdf55c8-bnd4j   0/1     ContainerCreating   0          3m43s
scoring-app-94fdf55c8-g7hlz   0/1     ImagePullBackOff    0          3m43s


In [34]:
!kubectl logs scoring-app-94fdf55c8-g7hlz

Error from server (BadRequest): container "scoring-app" in pod "scoring-app-94fdf55c8-g7hlz" is waiting to start: trying and failing to pull image


In [31]:
!kubectl describe pods

Name:           aks-ssh-589f4659c5-rdq97
Namespace:      default
Priority:       0
Node:           aks-nodepool1-21996804-4/10.240.0.4
Start Time:     Wed, 17 Jul 2019 06:33:16 +0000
Labels:         pod-template-hash=589f4659c5
                run=aks-ssh
Annotations:    <none>
Status:         Running
IP:             10.244.4.3
Controlled By:  ReplicaSet/aks-ssh-589f4659c5
Containers:
  aks-ssh:
    Container ID:   docker://7f475f1306f46f41c74510b2e880a34df6d0f1c564a4810ad33dc86792a27177
    Image:          debian
    Image ID:       docker-pullable://debian@sha256:903779f30a7ee46937bfb21406f125d5fdace4178074e1cc71c49039ebf7f48f
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Wed, 17 Jul 2019 06:33:54 +0000
    Ready:          True
    Restart Count:  0
    Environment:
      KUBERNETES_PORT_443_TCP_ADDR:  cambridgec-cambridge-43f949-773a2dc2.hcp.chinaeast2.cx.prod.service.azk8s.cn
      KUBERNETES_PORT:               tcp://cambridg



Name:           scoring-app-94fdf55c8-6tppl
Namespace:      default
Priority:       0
Node:           aks-nodepool1-21996804-3/10.240.0.7
Start Time:     Wed, 17 Jul 2019 09:38:58 +0000
Labels:         app=scoring-app
                pod-template-hash=94fdf55c8
Annotations:    <none>
Status:         Pending
IP:             10.244.1.5
Controlled By:  ReplicaSet/scoring-app-94fdf55c8
Containers:
  scoring-app:
    Container ID:   
    Image:          pjh177787/cambridgesp:latest
    Image ID:       
    Port:           433/TCP
    Host Port:      0/TCP
    State:          Waiting
      Reason:       ImagePullBackOff
    Ready:          False
    Restart Count:  0
    Environment:
      MOUNT_DIR:                     /data
      LB_LIBRARY_PATH:               $LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib
      DP_DISABLE_HEALTHCHECKS:       xids
      STORAGE_MODEL_DIR:             models
      SUBSCRIPTION_ID:               43f94930-ca31-4ddc-91c2-2f1be4f0e8f3
    

Expose the flask-app in the kubernetes cluster. This will open a public endpoint.

In [ ]:
!kubectl expose deployment flask-app --type="LoadBalancer"

Run `!watch kubectl get services` and wait until the external ip goes from pending to being realized. It can take some time.

NOTE: If the following command is run without the external ip being realized, an error will be thrown. 

In [ ]:
external_ip = !kubectl get services -o=jsonpath={.items[*].status.loadBalancer.ingress[0].ip}
external_ip = external_ip[0]

Since we'll use the `external_ip` later on, save it to the dot-env file.

In [ ]:
set_key(env_path, "AKS_EXTERNAL_IP", external_ip)

### Test that the deployment works end-to-end

Set the name of the new test video.

In [ ]:
new_video_name = "aks_test_orangutan.mp4"

Make a copy the old `orangutan.mp4` video but named with the `<new_video_name>`. 

In [ ]:
!cp data/orangutan.mp4 data/{new_video_name}

Use `curl` to hit the endpoint of the kubernetes cluster we just deployed.

In [ ]:
!curl {external_ip}":8080/process?video_name="{new_video_name}

Inspect your kubernetes cluster to see that the process is running. You can use the commands below to do so. Alternatively, you can also inspect the blob storage container to see that the images are being created.

When the video completes, you can play the video file directly from your mounted blob container:

In [ ]:
%%HTML
<video width="320" height="240" controls>
  <source src="data/aks_test_orangutan/aks_test_orangutan_processed.mp4" type="video/mp4">
</video>

### Basic Kubectl usage
You can use kubectl to perform basic monitoring. Use the following commands:
```bash
# monitor pods
!kubectl get pods

# print logs from a pod (<pod-name> can be found when calling 'get pods')
!kubectl logs <pod-name>

# check all services running on the cluster
!kubectl get services

# delete a service
!kubectl delete services <service-name>

# delete a deployment
!kubectl delete -f scoring_app_deployment.json
!kubectl delete -f flask_app_deployment.json
```

### Monitor in kubernetes dashboard
You can use the Kubernetes dashboard to monitor the cluster using the following commands:

```bash
# use the kube_dashboard_access.yaml to create a deployment
!kubectl create -f kube_dashboard_access.yaml

# use this command to browse
!az aks browse -n {get_key(env_path, "AKS_CLUSTER")} -g {get_key(env_path, "RESOURCE_GROUP")}
```

If you're not able to access the dashboard, follow the instructions [here](https://blog.tekspace.io/kubernetes-dashboard-remote-access/).

### Additional commands for AKS

Scale your AKS cluster:

```bash 
!az aks scale \
    --name {get_key(env_path, "AKS_CLUSTER")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --node-count 10
```

Scale your deployment:
```bash
!kubectl scale deployment.apps/aks-app --replicas=10
```

---

Continue to the next [notebook](/notebooks/05_deploy_logic_app.ipynb).